In [7]:
import numpy as np
import Algorithms as alg
import pandas as pd
import wfdb
import os

os.makedirs('dataframes', exist_ok=True)


In [8]:
path = 'database\RECORDS'

with open(path, 'r') as file:
    tapes = [int(line.strip()) for line in file]

In [9]:
number_of_symbols = {}

for tape in tapes:
    filename = 'database/'+ str(tape)
    record = wfdb.rdrecord(filename)

    ann= wfdb.rdann(filename, 'atr')

    for symbol in ann.symbol:
        if symbol in number_of_symbols:
            number_of_symbols[symbol] += 1
        else:
            number_of_symbols[symbol] = 1
        

unique_symbols = list(number_of_symbols.keys())

print(unique_symbols)

['+', 'N', 'A', 'V', '~', '|', 'Q', '/', 'f', 'x', 'F', 'j', 'L', 'a', 'J', 'R', '[', '!', ']', 'E', 'S', '"', 'e']


In [10]:
print("Number of each symbol: \n  ->", number_of_symbols)
print("All unique symbols: \n  ->", unique_symbols)

number_of_all_symbols = 0
for s in unique_symbols:
    number_of_all_symbols += number_of_symbols[s]

print("Number of all symbols: ", number_of_all_symbols)

target_symbols = ['N', 'L', 'R', 'A', 'V', '/'] 

number_of_target_symbols = 0
for s in target_symbols:
    number_of_target_symbols += number_of_symbols[s]

print("Target symbols: \n  ->", target_symbols)
print("Number of all target symbols: ", number_of_target_symbols)

Number of each symbol: 
  -> {'+': 1291, 'N': 75052, 'A': 2546, 'V': 7130, '~': 616, '|': 132, 'Q': 33, '/': 7028, 'f': 982, 'x': 193, 'F': 803, 'j': 229, 'L': 8075, 'a': 150, 'J': 83, 'R': 7259, '[': 6, '!': 472, ']': 6, 'E': 106, 'S': 2, '"': 437, 'e': 16}
All unique symbols: 
  -> ['+', 'N', 'A', 'V', '~', '|', 'Q', '/', 'f', 'x', 'F', 'j', 'L', 'a', 'J', 'R', '[', '!', ']', 'E', 'S', '"', 'e']
Number of all symbols:  112647
Target symbols: 
  -> ['N', 'L', 'R', 'A', 'V', '/']
Number of all target symbols:  107090


In [11]:
df = pd.DataFrame(index=tapes, columns=['ECG', 'True_peak', 'Peak_type'])

for index in df.index:
    filename = 'database/'+ str(index)
    record = wfdb.rdrecord(filename)

    ecg = list(map(lambda x: x[0], record.p_signal))
    ann= wfdb.rdann(filename, 'atr')
    # annotations is a list of tuples locations and annotation letter [(77, 'N'), (370, 'N'), (662, 'N'), ...]
    annotations = list(filter(lambda x: x[1] in unique_symbols, zip(ann.sample, ann.symbol)))
    x2, a2 = zip(*annotations)

    df.at[index, 'ECG'] = ecg
    df.at[index, 'True_peak'] = np.array(x2)  
    df.at[index, 'Peak_type'] = np.array(a2)  

name = 'dataframes\df_mitdb.pkl'
df.to_pickle(name)
print(f"Dataframe saved as {name}")

Dataframe saved as dataframes\df_mitdb.pkl


In [12]:
df_algorithms = pd.DataFrame(index=tapes, columns=['Alg1', 'Alg2', 'Alg3', 'Alg4', 'Alg5'])

for index in df_algorithms.index:
    filename = 'database/'+ str(index)
    record = wfdb.rdrecord(filename)

    ecg = list(map(lambda x: x[0], record.p_signal))

    df_algorithms.at[index, 'Alg1'] = np.array(alg.alg1_spanish(ecg))
    df_algorithms.at[index, 'Alg2'] = np.array(alg.alg4_polish_20210222(ecg))
    df_algorithms.at[index, 'Alg3'] = np.array(alg.alg3_iranian(ecg))
    df_algorithms.at[index, 'Alg4'] = np.array(alg.alg5_pan_tompkins(ecg))
    df_algorithms.at[index, 'Alg5'] = np.array(alg.Alg5_Turkish(ecg))


name = 'dataframes\df_all_algorithms.pkl'
df_algorithms.to_pickle(name)
print(f"Dataframe saved as {name}")


Dataframe saved as dataframes\df_all_algorithms.pkl
